In [128]:
import pandas as pd

customer_df = pd.read_csv("customer.csv",header=0)
customer_address_df = pd.read_csv("customer_address.csv",header=0)


In [85]:
customer_df.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car
0,1,Laraine,Medendorp,F,10/12/1953,Executive Secretary,Health,Mass Customer,N,Yes
1,2,Eli,Bockman,Male,12/16/1980,Administrative Officer,Financial Services,Mass Customer,N,Yes
2,3,Arlin,Dearle,Male,1/20/1954,Recruiting Manager,Property,Mass Customer,N,Yes
3,4,Talbot,NaN,Male,10/3/1961,NaN,IT,Mass Customer,N,No
4,5,Sheila-kathryn,Calton,Female,5/13/1977,Senior Editor,NaN,Affluent Customer,N,Yes


In [86]:
customer_address_df.head()

,address_id,customer_id,address,postcode,state,country
0,1,1,060 Morning Avenue,2016,New South Wales,Australia
1,2,2,6 Meadow Vale Court,2153,New South Wales,Australia
2,3,4,0 Holy Cross Court,4211,QLD,Australia
3,4,5,17979 Del Mar Point,2448,New South Wales,Australia
4,5,6,9 Oakridge Court,3216,VIC,Australia


In [129]:
result=[]
for c in customer_df.index:
    address_df = customer_address_df[customer_address_df['customer_id'] == customer_df['customer_id'][c]]
    list_address=[]
    for a in address_df.index:
        list_address.append(f"{customer_address_df['address'][a]},{customer_address_df['state'][a]},{customer_address_df['postcode'][a]}")
    result.append(list_address)
    
customer_df['addresses']=result

In [130]:
customer_df[customer_df['customer_id']==4015]['addresses']

4014    [6860 Green Ridge Avenue,NSW,2126, 85 Straubel...
Name: addresses, dtype: object

In [131]:
customer_df.rename(columns={'customer_id':'unique_id'}, inplace=True)
customer_df.head()

,unique_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,addresses
0,1,Laraine,Medendorp,F,10/12/1953,Executive Secretary,Health,Mass Customer,N,Yes,"[060 Morning Avenue,New South Wales,2016]"
1,2,Eli,Bockman,Male,12/16/1980,Administrative Officer,Financial Services,Mass Customer,N,Yes,"[6 Meadow Vale Court,New South Wales,2153]"
2,3,Arlin,Dearle,Male,1/20/1954,Recruiting Manager,Property,Mass Customer,N,Yes,[]
3,4,Talbot,NaN,Male,10/3/1961,NaN,IT,Mass Customer,N,No,"[0 Holy Cross Court,QLD,4211]"
4,5,Sheila-kathryn,Calton,Female,5/13/1977,Senior Editor,NaN,Affluent Customer,N,Yes,"[17979 Del Mar Point,New South Wales,2448]"


In [132]:
import math
from nicknames import NickNamer

nn = NickNamer()

list_first_name_nicks=[]

for index, row in customer_df.iterrows():
    first_name = row['first_name']
    nicks = nn.nicknames_of(first_name)
    list_first_name_nicks.append(list(nicks))

customer_df['nicks_first_name'] = list_first_name_nicks

In [165]:
import splink.comparison_library as cl
from splink import DuckDBAPI, Linker, SettingsCreator, block_on, splink_datasets

settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        {
            "output_column_name": "first_name",
            "comparison_levels": [
                {
                    "sql_condition": "\"first_name_l\" IS NULL OR \"first_name_r\" IS NULL",
                    "label_for_charts": "first_name is NULL",
                    "is_null_level": True
                },
                {
                    "sql_condition": "\"first_name_l\" = \"first_name_r\"",
                    "label_for_charts": "Exact match on first_name"
                },
                {
                    "sql_condition": "jaro_winkler_similarity(\"first_name_l\", \"first_name_r\") >= 0.9",
                    "label_for_charts": "Jaro-Winkler distance of first_name >= 0.9"
                },
                {
                    "sql_condition": "jaro_winkler_similarity(\"first_name_l\", \"first_name_r\") >= 0.8",
                    "label_for_charts": "Jaro-Winkler distance of first_name >= 0.8"
                },
                {
                    "sql_condition": "list_contains(nicks_first_name_l, first_name_r) OR list_contains(nicks_first_name_r, first_name_l)",
                    "label_for_charts": "Nick Name comparision for first_name"
                },
                {
                    "sql_condition": "ELSE",
                    "label_for_charts": "All other comparisons"
                }
            ],
            "comparison_description": "nick name and jaro_winkler at thresholds 0.9, 0.8 vs. anything else"
        },        
        cl.ExactMatch("last_name"),
        cl.ExactMatch("gender"),
        cl.DateOfBirthComparison(
            "DOB",
            input_is_string=True,),
        cl.ArrayIntersectAtSizes("addresses", [2, 1]),
    ],
    blocking_rules_to_generate_predictions=[
        block_on("first_name", "last_name", "DOB"),
        block_on("first_name", "DOB"),
        block_on("DOB", "addresses"),
    ],
    max_iterations=2,    
)


In [166]:
linker = Linker(customer_df, settings, DuckDBAPI(), set_up_basic_logging=False)
df_predict = linker.inference.predict()
df_e = df_predict.as_pandas_dataframe()
df_e


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'first_name':
    m values not fully trained
Comparison: 'first_name':
    u values not fully trained
Comparison: 'last_name':
    m values not fully trained
Comparison: 'last_name':
    u values not fully trained
Comparison: 'gender':
    m values not fully trained
Comparison: 'gender':
    u values not fully trained
Comparison: 'DOB':
    m values not fully trained
Comparison: 'DOB':
    u values not fully trained
Comparison: 'addresses':
    m values not fully trained
Comparison: 'addresses':
    u values not fully trained
The 'probability_two_random_records_match' setting has been set to the default value (0.0001). 
If this is not the desired behaviour, either: 
 - assign a value for `probability_two_random_records_match` in

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,nicks_first_name_l,nicks_first_name_r,gamma_first_name,last_name_l,...,gender_l,gender_r,gamma_gender,DOB_l,DOB_r,gamma_DOB,addresses_l,addresses_r,gamma_addresses,match_key
0,-18.287568,0.000003,273,4011,Nevile,Nevile,[],[],4,Abraham,...,Male,Female,0,4/22/1963,4/22/1963,-1,"[8 Texas Plaza,VIC,3081]",[],0,1
1,-3.287568,0.092897,343,4015,Nicole,Nikki,"[nikki, nicky, cole, nicki, nole]",[],0,Johananoff,...,Female,Female,1,8/14/1957,8/14/1957,-1,"[6860 Green Ridge Avenue,NSW,2126, 85 Straubel...","[6860 Green Ridge Avenue,NSW,2126, 85 Straubel...",2,2
2,-3.287568,0.092897,273,4010,Nevile,Nevile,[],[],4,Abraham,...,Male,Male,1,4/22/1963,4/22/1963,-1,"[8 Texas Plaza,VIC,3081]",[],0,1
3,-3.287568,0.092897,4010,4011,Nevile,Nevile,[],[],4,Abe,...,Male,Female,0,4/22/1963,4/22/1963,-1,[],[],0,0
4,-3.287568,0.092897,4013,4016,Nikki,Cole,[],[colie],0,Johananoff,...,Female,Female,1,8/14/1957,8/14/1957,-1,[],[],0,2
5,-18.287568,0.000003,4014,4016,Nicky,Cole,[],[colie],0,Johana,...,Female,Female,1,8/14/1957,8/14/1957,-1,[],[],0,2
6,-3.287568,0.092897,2194,4004,Johnette,Johnette,[],[],4,O'Fergus,...,Female,Female,1,9/10/1958,9/10/1958,-1,"[9045 Kings Avenue,QLD,4153]","[02 Fair Oaks Lane,VIC,3977]",0,1
7,-3.287568,0.092897,4013,4015,Nikki,Nikki,[],[],4,Johananoff,...,Female,Female,1,8/14/1957,8/14/1957,-1,[],"[6860 Green Ridge Avenue,NSW,2126, 85 Straubel...",0,1
8,-18.287568,0.000003,4013,4014,Nikki,Nicky,[],[],0,Johananoff,...,Female,Female,1,8/14/1957,8/14/1957,-1,[],[],0,2


In [173]:
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    df_predict)
clusters.as_pandas_dataframe(limit=5)

,cluster_id,unique_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,addresses,nicks_first_name
0,1,1,Laraine,Medendorp,F,10/12/1953,Executive Secretary,Health,Mass Customer,N,Yes,"[060 Morning Avenue,New South Wales,2016]",[]
1,2,2,Eli,Bockman,Male,12/16/1980,Administrative Officer,Financial Services,Mass Customer,N,Yes,"[6 Meadow Vale Court,New South Wales,2153]",[]
2,4,4,Talbot,None,Male,10/3/1961,None,IT,Mass Customer,N,No,"[0 Holy Cross Court,QLD,4211]",[]
3,5,5,Sheila-kathryn,Calton,Female,5/13/1977,Senior Editor,None,Affluent Customer,N,Yes,"[17979 Del Mar Point,New South Wales,2448]",[]
4,6,6,Curr,Duckhouse,Male,9/16/1966,None,Retail,High Net Worth,N,Yes,"[9 Oakridge Court,VIC,3216]",[]


In [174]:
df = clusters.as_pandas_dataframe()

In [175]:
df['cluster_id'].nunique()

4009

In [176]:
len(df['cluster_id'])

4016

In [177]:
df_group = df.groupby('cluster_id')
df_res = df_group.agg({'unique_id': 'nunique'}).reset_index()
df_res = df_res[df_res['unique_id']>1]
len(df_res)

3

In [178]:
df_out = pd.merge(df, df_res, on="cluster_id", how="right")
df_out.head()

,cluster_id,unique_id_x,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,addresses,nicks_first_name,unique_id_y
0,273,273,Nevile,Abraham,Male,4/22/1963,Computer Systems Analyst IV,None,Mass Customer,N,No,"[8 Texas Plaza,VIC,3081]",[],3
1,273,4011,Nevile,Abe,Female,4/22/1963,Computer Systems Analyst IV,None,Mass Customer,N,No,[],[],3
2,273,4010,Nevile,Abe,Male,4/22/1963,Computer Systems Analyst IV,None,Mass Customer,N,No,[],[],3
3,343,343,Nicole,Johananoff,Female,8/14/1957,Recruiter,Property,Affluent Customer,N,No,"[6860 Green Ridge Avenue,NSW,2126, 85 Straubel...","[nikki, nicky, cole, nicki, nole]",5
4,343,4013,Nikki,Johananoff,Female,8/14/1957,Recruiter,Property,Affluent Customer,N,No,[],[],5


In [179]:
df_out.to_csv("result.csv", header=True)
